<a href="https://colab.research.google.com/github/Erenere/ICGrLFqa1TiymKPG/blob/main/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# -*- coding: utf-8 -*-

!pip install -q sklearn # if needed

# Commented out IPython magic to ensure Python compatibility.
# %tensorflow_version 2.x

from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
from sklearn.model_selection import train_test_split
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow import keras

df=pd.read_csv("/content/ACME-HappinessSurvey2020.csv")

dftrain = df.copy()
dfeval = df.copy()

y_train = dftrain.pop('Y')
y_eval = dfeval.pop('Y')
for y in y_train:
  y=int(y)
for y in y_eval:
  y=int(y)
# dftrain.head()

def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=["accuracy"])
  return model

normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(dftrain))

dnn_model = build_and_compile_model(normalizer)
#dnn_model.summary()

# %%time
history = dnn_model.fit(
    dftrain, y_train,
    validation_split=0.2,
    verbose=0, epochs=100)

test_results = {}
test_results['dnn_model'] = dnn_model.evaluate(dfeval, y_eval, verbose=1)
# test_results

dnn_model.save('dnn_model')

sample_size=126
pred=dnn_model.predict_classes(dfeval[:sample_size])

real=y_eval[:sample_size]
score=0
for i in range(sample_size):
  if pred[i]==real[i]:
    score+=1
print(f'score: {score/sample_size}')

4/4 [==============================] - 0s 3ms/step - loss: 1.0344 - accuracy: 0.8254
INFO:tensorflow:Assets written to: dnn_model/assets
score: 0.8253968253968254


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [8]:
# pearson's correlation feature selection for numeric input and numeric output
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

fs = SelectKBest(score_func=f_regression, k=1)
# apply feature selection
X_selected = fs.fit_transform(dftrain, y_train)
print(X_selected)
# Most important feature -> X1

[[3]
 [3]
 [5]
 [5]
 [5]
 [5]
 [3]
 [5]
 [4]
 [4]
 [3]
 [4]
 [5]
 [4]
 [4]
 [3]
 [5]
 [5]
 [5]
 [4]
 [4]
 [4]
 [4]
 [5]
 [4]
 [3]
 [3]
 [3]
 [3]
 [5]
 [5]
 [4]
 [3]
 [3]
 [4]
 [4]
 [5]
 [5]
 [5]
 [4]
 [5]
 [4]
 [5]
 [4]
 [3]
 [5]
 [5]
 [1]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [4]
 [4]
 [5]
 [4]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [4]
 [4]
 [3]
 [4]
 [5]
 [5]
 [4]
 [4]
 [4]
 [5]
 [5]
 [3]
 [4]
 [5]
 [5]
 [5]
 [4]
 [3]
 [4]
 [5]
 [4]
 [5]
 [5]
 [4]
 [5]
 [3]
 [3]
 [5]
 [5]
 [5]
 [3]
 [5]
 [4]
 [5]
 [4]
 [3]
 [5]
 [5]
 [5]
 [4]
 [4]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [5]
 [4]
 [5]
 [5]
 [5]
 [3]
 [5]
 [4]
 [5]
 [4]
 [5]
 [5]
 [5]
 [4]
 [5]]


In [9]:
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier(n_estimators=10)
model.fit(dftrain, y_train)
print(model.feature_importances_)
# X1 is the most important, X2 and X3 have also high importance

[0.19411442 0.18979693 0.19306574 0.1576853  0.13934504 0.12599258]
